In [1]:
#link for probability distribution functions https://docs.scipy.org/doc/scipy-0.16.1/reference/stats.html
import numpy as np
from scipy.stats import norm
from scipy.stats import t
'''
a: alpha (nível de confiança equivale a 1-a)
e: epsilon (margem de erro, da mesma unbidade de medida do parâmetro)
N: Tamanho da população
sd: desvio-padrão envolvido no cálculo
sdSampleSize: tamanho da amostra da qual obteve-se sd
isToProportion: se True, estuda-se o tamanho amostral para proporção. 
Se False, esrtuda a média em geral. 
'''
def getSampleSize(a = .05, e = -1, N=-1, sd = -1, sdSampleSize = -1, isToProportion=False):
    m=sdSampleSize#TAMANHO DA AMOSTRA DA QUAL PROVEIO sd
    quant_m = -1; text = ''
    normalCondition = True
    if sdSampleSize>0 or (N>0 and sdSampleSize==N): 
        normalCondition = False
    if normalCondition or isToProportion:#sd is the parameter value
        # quant_m = round(norm.ppf(1-a/2), 2)
        quant_m = norm.ppf(1-a/2)
    else:#sd is an  estimate of the sd parameter
        # quant_m = round(t.ppf((1-a/2), df=(m-1)), 3)#QUANTIL DA T-STUDENT
        quant_m = t.ppf((1-a/2), df=(m-1))#QUANTIL DA T-STUDENT
        text = ", df=" + str(m-1)
    
    n0 = (quant_m*sd/e)**2
    n = n0 if N<0 else (n0*N)/(n0+N-1)  
    n=round(n)#TAMANHO DA AMOSTRA
    print("quant_m=" + str(quant_m) + ", n=" + str(n) + text)
    return(n)


In [2]:
#População de tamanho infinito (N=-1), desvio padrão popul conhecido (sdSampleSize = -1):
getSampleSize(a = .05, e = .1, N=-1, sd = 1, sdSampleSize = -1, isToProportion=False)
#População de tamanho finito (N=10), desvio padrão popul desconhecido (sdSampleSize = -1):
getSampleSize(a = .05, e = .1, N=10, sd = 1, sdSampleSize = 5, isToProportion=False)
#População de tamanho infinito (N=-1), abordagem ousada (p_obs = .8), para proporção:
getSampleSize(a = .05, e = .02, N=-1, sd = np.sqrt(.8*.2), sdSampleSize = -1, 
              isToProportion=True)
#População de tamanho finito (N=10), abordagem conservadora (p_obs = .5), para proporção:
getSampleSize(a = .05, e = .1, N=10, sd = np.sqrt(.9*.1), sdSampleSize = -1, 
              isToProportion=True)

quant_m=1.959963984540054, n=384
quant_m=2.7764451051977987, n=10, df=4
quant_m=1.959963984540054, n=1537
quant_m=1.959963984540054, n=8


8

In [3]:
#Exemplo: Conjunto de dados de manutenção
import pandas as pd
#Ler o conjunto
BD01 = pd.read_csv("../data/BD01.csv", delimiter=";")#, dec = ",", encoding="UTF-8", sep=";", quote="")
# BD01 = pd.read_csv("../input/bd01-data/BD01.csv", delimiter=";")#, dec = ",", encoding="UTF-8", sep=";", quote="")
#visualizar o conjunto
display(BD01.head(10))
print('BD01.shape:', BD01.shape)

,Fornecedor,Degradacao,TempoFalha,nReincidenciaFalhas
0,B,Alto,1.957318,2
1,B,Alto,0.364677,13
2,B,Medio,2.640282,1
3,B,Alto,1.187134,0
4,A,Baixo,18.337973,4
5,A,Baixo,12.536926,21
6,A,Baixo,14.411633,22
7,A,Baixo,18.107021,60
8,A,Baixo,14.849921,20
9,A,Baixo,46.790521,13


BD01.shape: (579, 4)


In [4]:
N = len(BD01)
a = .05
sd = BD01['TempoFalha'].std(ddof=0)
e = .1*sd#hora
n = getSampleSize(a = a, e = e, N=N, sd = sd, sdSampleSize = -1, isToProportion=False)

quant_m=1.959963984540054, n=231


In [5]:
.6*N

347.4

In [6]:
# referência: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html
import pandas as pd
# amostragem aleatoria simples - AASimples
def getSimpleRandomSample(dataFrame, sampleSize):
    # N = len(dataFrame) #tamanho da população
    # n = sampleSize # tamanho da amostra
    # amostra aleatória de índices
    # sampleIndexes = sample(x = 1:N, size = n, replace = FALSE)
    # separando a amostra aleatória correspondente aos índices sorteados
    #sampleData = dataFrame[sampleIndexes, ]
    sampleData = dataFrame.sample(n=sampleSize, replace=False)
    return (sampleData)

In [7]:
# amostragem aleatoria sistemática - AASistemática
# import numpy as np

def getSystematicRandomSample(dataFrame, sampleSize):
    N = len(dataFrame)  # tamanho da população
    n = sampleSize  # tamanho da amostra
    ti = 0
    tf = (N-1)  # mínimo e máximo do intervalo de chaves
    d = round((tf - ti + 1)/n)  # distância entre índices subsequentes
    # primeiro índice sorteado aleatoriamente (como na AASimples)
    i0 = np.random.default_rng().integers(low=ti, high=ti+d)
    # sorteando índices a partir de i0, distanciados por d
    sampleIndexes = i0 + list(range(0, N, d))
    # contornando o problema de índice maior que o máximo possível
    #sampleIndexes[sampleIndexes > tf] = sampleIndexes[sampleIndexes > tf] - tf + ti - 1
    # separando a amostra aleatória correspondente aos índices sorteados
    sampleData = dataFrame.loc[sampleIndexes, ]
    print("*********** sampleData ***********")
    display(sampleData)
    return (sampleData)


In [8]:
# amostragem aleatoria estratificada proporcional - AAEstrProp
def getProportionalStratifiedRandomSample(dataFrame, sampleSize, stratumVariable, 
                                          subSampleFunction = getSystematicRandomSample):
  # distribuição da variável de estrato na população
  n = sampleSize
  probDist = dataFrame[stratumVariable].value_counts(normalize=True) # View(probDist)
  print('*************** probDist ***************')
  display(probDist)
  # ns = round(n*probDist.values);  # View(ns)
  stratums = probDist.index.to_list()
  nStratums = len(stratums)
  sampleData = pd.DataFrame()
  for i in range(nStratums):
    stratum_i = stratums[i]
    # View(dataFrame_i, title = stratum_i)
    dataFrame_i = dataFrame[dataFrame[stratumVariable] == stratum_i]
    N_i = len(dataFrame_i)  # tamanho da iª população
    n_i = round(n*probDist.values[i])  # tamanho da iª amostra
    # View(sampleData_i, title = paste("amostra_", stratum_i) )
    sampleData_i = subSampleFunction(dataFrame= dataFrame_i, sampleSize= n_i)
    #sampleData_i = getSystematicRandomSample(dataFrame= dataFrame_i, sampleSize= n_i)
    sampleData = pd.concat([sampleData, sampleData_i], axis = 0)
  print('********** sample **********')
  display(sampleData)
  return (sampleData)

In [10]:
N = len(BD01)
a = .05
sd = BD01['TempoFalha'].std(ddof=0)
e = .1*sd#hora
print('e='+str(e))
n = getSampleSize(a = a, e = e, N=N, sd = sd, sdSampleSize = -1, isToProportion=False)

aaProp_data = getProportionalStratifiedRandomSample(dataFrame=BD01, sampleSize=n, 
                                                    stratumVariable='Fornecedor', #"Degradacao",
                                                    subSampleFunction = getSimpleRandomSample)
#print(aaProp_data.head(n))
print("************** BD01.describe() **************")
display(BD01.describe())
print("************** aaSimples_data.describe() **************")
display(aaProp_data.describe())

e=2.199295481414015
quant_m=1.959963984540054, n=231
*************** probDist ***************


Fornecedor
A    0.609672
B    0.390328
Name: proportion, dtype: float64

********** sample **********


,Fornecedor,Degradacao,TempoFalha,nReincidenciaFalhas
507,A,Baixo,11.842930,1
265,A,Baixo,13.990066,3
256,A,Baixo,14.891575,4
42,A,Baixo,20.749899,5
464,A,Baixo,45.784748,14
...,...,...,...,...
135,B,Alto,0.714799,56
536,B,Medio,4.366681,12
532,B,Alto,2.038045,18
557,B,Alto,0.991718,3


************** BD01.describe() **************


,TempoFalha,nReincidenciaFalhas
count,579.000000,579.000000
mean,20.250725,18.792746
std,22.011972,19.226708
min,0.036110,0.000000
25%,2.825574,3.000000
50%,14.318828,14.000000
75%,30.370120,28.000000
max,121.861409,125.000000


************** aaSimples_data.describe() **************


,TempoFalha,nReincidenciaFalhas
count,231.000000,231.000000
mean,20.373447,18.212121
std,21.944410,18.549652
min,0.091826,0.000000
25%,2.495486,3.000000
50%,13.646376,14.000000
75%,32.253459,26.000000
max,111.997702,101.000000


In [16]:
print('O tempo médio até a falha é de '+\
    str(round(aaProp_data.describe().loc['mean','TempoFalha'], 2))+\
    ' horas, com margem de erro de '+\
    str(round(e, 2)) + " horas \n para mais ou para menos.")

O tempo médio até a falha é de 20.37 horas, com margem de erro de 2.2 horas 
 para mais ou para menos


In [26]:
# import numpy as np
pTime = BD01.describe().loc['mean', 'TempoFalha']
eTime = aaProp_data.describe().loc['mean', 'TempoFalha']
diferencaRelativaTime = np.abs(pTime - eTime)/pTime
print('diferencaRelativaTime = ' + str(diferencaRelativaTime))

diferencaRelativaTime = 0.05598667935111227


In [28]:
#intervalo de confiança
xBarra = aaProp_data.describe().loc['mean', 'TempoFalha']
print('['+str(xBarra - e)+ ', '+str(xBarra + e)+']')

[19.061780153932304, 23.460371116760335]


: 